In [3]:
%matplotlib inline
import pickle
import datetime
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
from functools import reduce
from sklearn import ensemble
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import itertools

In [5]:
df = pd.read_csv("cleaned_dataset.csv", header=0, low_memory=False)

cols = list(["country"]) + list(df.columns[1:])

df.columns = cols

df

,country,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_k,eurozone
0,ABW,Aruba,Aruban Guilder,1950,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,ABW,Aruba,Aruban Guilder,1951,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,ABW,Aruba,Aruban Guilder,1952,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,ABW,Aruba,Aruban Guilder,1953,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,ABW,Aruba,Aruban Guilder,1954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,ABW,Aruba,Aruban Guilder,1955,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
6,ABW,Aruba,Aruban Guilder,1956,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7,ABW,Aruba,Aruban Guilder,1957,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8,ABW,Aruba,Aruban Guilder,1958,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
9,ABW,Aruba,Aruban Guilder,1959,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [6]:
# df.Unit.value_counts()
# df.variable.unique()
df.isnull().sum()

country              0
country              0
currency_unit        0
year                 0
rgdpe             2391
rgdpo             2391
pop               2391
emp               3586
avh               8511
hc                3963
ccon              2391
cda               2391
cgdpe             2391
cgdpo             2391
ck                2421
ctfp              5877
cwtfp             5877
rgdpna            2391
rconna            2391
rdana             2391
rkna              2421
rtfpna            5877
rwtfpna           5877
labsh             4348
delta             2421
xr                2391
pl_con            2391
pl_da             2391
pl_gdpo           2391
i_cig             2391
i_xm              2391
i_xr              2391
i_outlier         2391
cor_exp          11220
statcap          10324
csh_c             2391
csh_i             2391
csh_g             2391
csh_x             2391
csh_m             2391
csh_r             2391
pl_c              2391
pl_i              2391
pl_g       

In [4]:
df_final = pd.pivot_table(df, values='Value', index=['country', 'Date'], columns=['variable'])
df_final

variable       Average annual hours worked by persons engaged  \
country  Date                                                   
Albania  1970                                             NaN   
         1971                                             NaN   
         1972                                             NaN   
         1973                                             NaN   
         1974                                             NaN   
         1975                                             NaN   
         1976                                             NaN   
         1977                                             NaN   
         1978                                             NaN   
         1979                                             NaN   
         1980                                             NaN   
         1981                                             NaN   
         1982                                             NaN   
         1983                                             NaN   
         1984                                             NaN   
         1985                                             NaN   
         1986                                             NaN   
         1987                                             NaN   
         1988                                             NaN   
         1989                                             NaN   
         1990                                             NaN   
         1991                                             NaN   
         1992                                             NaN   
         1993                                             NaN   
         1994                                             NaN   
         1995                                             NaN   
         1996                                             NaN   
         1997                                             NaN   
         1998                                             NaN   
         1999                                             NaN   
...                                                       ...   
Zimbabwe 1982                                             NaN   
         1983                                             NaN   
         1984                                             NaN   
         1985                                             NaN   
         1986                                             NaN   
         1987                                             NaN   
         1988                                             NaN   
         1989                                             NaN   
         1990                                             NaN   
         1991                                             NaN   
         1992                                             NaN   
         1993                                             NaN   
         1994                                             NaN   
         1995                                             NaN   
         1996                                             NaN   
         1997                                             NaN   
         1998                                             NaN   
         1999                                             NaN   
         2000                                             NaN   
         2001                                             NaN   
         2002                                             NaN   
         2003                                             NaN   
         2004                                             NaN   
         2005                                             NaN   
         2006                                             NaN   
         2007                                             NaN   
         2008                                             NaN   
         2009                                             NaN   
         2010                      

In [26]:
# df_final.groupby(df_final.index.country)
# df_final.index
df_final.groupby(level = 0).sum()

variable,Average annual hours worked by persons engaged,Average depreciation rate of the capital stock,Capital stock at constant 2011 national prices (in mil. 2011US$),Capital stock at current PPPs (in mil. 2011US$),Correlation between expenditure shares of the country and the US (benchmark observations only),"Exchange rate, national currency/USD (market+estimated)",Expenditure-side real GDP at chained PPPs (in mil. 2011US$),Expenditure-side real GDP at current PPPs (in mil. 2011US$),"Human capital index, based on years of schooling and returns to education",Number of persons engaged (in millions),...,Share of household consumption at current PPPs,Share of labour compensation in GDP at current national prices,Share of merchandise exports at current PPPs,Share of merchandise imports at current PPPs,Share of residual trade and GDP statistical discrepancy at current PPPs,"Statistical capacity indicator (source: World Bank, developing countries only)",TFP at constant national prices (2011=1),TFP level at current PPPs (USA=1),Welfare-relevant TFP at constant national prices (2011=1),Welfare-relevant TFP levels at current PPPs (USA=1)
country,,,,,,,,,,,,,,,,,,,,,
Albania,NaN,1.502417,1.901380e+06,1.431986e+06,1.233308,2294.044260,5.774961e+05,5.528554e+05,98.989487,40.571579,...,31.330087,NaN,1.803206,-5.097405,-0.791937,680.000000,NaN,NaN,NaN,NaN
Algeria,NaN,2.724273,3.706372e+07,3.396693e+07,0.398455,1417.027980,1.073869e+07,1.052090e+07,77.775273,238.955234,...,18.151505,NaN,10.387408,-5.227685,-1.281241,546.666667,NaN,NaN,NaN,NaN
Angola,NaN,2.344612,1.477279e+07,7.325376e+06,0.858669,821.755036,1.932520e+06,1.952713e+06,49.531906,180.718087,...,21.293242,NaN,12.654041,-4.222178,-5.619188,363.888889,NaN,NaN,NaN,NaN
Anguilla,NaN,1.429417,5.161534e+04,4.923667e+04,0.749573,109.186533,8.206847e+03,8.467264e+03,NaN,0.096721,...,20.138862,NaN,17.974763,-28.079120,2.066951,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,NaN,2.013324,1.864398e+05,7.268070e+04,1.421221,109.193069,3.707932e+04,3.534719e+04,NaN,0.289954,...,28.916969,NaN,11.942437,-35.346535,10.587764,273.888889,NaN,NaN,NaN,NaN
Argentina,119906.260315,2.953667,5.019525e+07,3.405789e+07,1.864898,44.271753,1.522260e+07,1.505007e+07,143.682501,650.955688,...,45.540375,31.252082,6.032953,-4.247980,-0.200087,756.111111,59.421454,40.907844,52.609086,39.312691
Armenia,5541.000000,0.761751,5.111674e+05,6.853216e+05,0.769572,7924.862670,3.034552e+05,2.974050e+05,67.379938,31.545736,...,15.813448,31.461901,1.116696,-3.312737,-1.103046,760.000000,14.834643,6.349390,15.114903,7.027433
Aruba,NaN,1.556646,5.201129e+05,4.732264e+05,0.671643,75.180000,1.020840e+05,1.022312e+05,NaN,0.905987,...,16.552303,27.164166,24.498388,-23.562995,-0.012081,NaN,NaN,NaN,NaN,NaN
Australia,125438.685593,2.951251,8.700029e+07,8.157516e+07,3.541396,68.658411,2.503020e+07,2.496892e+07,195.684692,399.797559,...,36.010713,39.917506,9.832243,-9.325630,-0.687960,NaN,52.175426,51.043769,46.385044,50.423813


In [28]:
df_final.groupby(level = 1).sum()

variable,Average annual hours worked by persons engaged,Average depreciation rate of the capital stock,Capital stock at constant 2011 national prices (in mil. 2011US$),Capital stock at current PPPs (in mil. 2011US$),Correlation between expenditure shares of the country and the US (benchmark observations only),"Exchange rate, national currency/USD (market+estimated)",Expenditure-side real GDP at chained PPPs (in mil. 2011US$),Expenditure-side real GDP at current PPPs (in mil. 2011US$),"Human capital index, based on years of schooling and returns to education",Number of persons engaged (in millions),...,Share of household consumption at current PPPs,Share of labour compensation in GDP at current national prices,Share of merchandise exports at current PPPs,Share of merchandise imports at current PPPs,Share of residual trade and GDP statistical discrepancy at current PPPs,"Statistical capacity indicator (source: World Bank, developing countries only)",TFP at constant national prices (2011=1),TFP level at current PPPs (USA=1),Welfare-relevant TFP at constant national prices (2011=1),Welfare-relevant TFP levels at current PPPs (USA=1)
Date,,,,,,,,,,,,,,,,,,,,,
1950,65027.380567,2.176731,1.683398e+07,1.397184e+07,NaN,471.138411,5.759000e+06,5.682570e+06,93.133320,692.149979,...,38.465489,29.500146,7.080185,-8.269707,0.564435,NaN,30.341282,30.034912,30.605343,30.534146
1951,72122.659407,2.432690,2.049290e+07,1.518738e+07,NaN,489.090871,6.198692e+06,6.153946e+06,101.221888,722.153745,...,41.865953,32.142009,8.540919,-10.831574,0.674794,NaN,31.913792,32.581192,32.381097,33.983664
1952,71823.122619,2.512752,2.167967e+07,1.644028e+07,NaN,492.370958,6.960203e+06,6.934346e+06,102.834673,740.121043,...,41.822119,32.818570,8.216746,-9.788494,0.319354,NaN,32.841127,32.886695,32.402054,33.495939
1953,74011.379033,2.634869,2.254113e+07,1.705410e+07,NaN,529.101044,7.369170e+06,7.347762e+06,106.956843,762.166873,...,43.314089,34.066587,8.015426,-9.513516,0.089607,NaN,34.464835,33.629777,33.991614,34.103700
1954,76396.274504,2.821846,2.348569e+07,1.746233e+07,NaN,539.351661,7.616047e+06,7.572851e+06,112.675801,776.684502,...,48.529213,35.753087,8.533697,-14.157545,0.536637,NaN,37.233424,36.197726,37.199428,37.580361
1955,76223.180409,3.011842,2.489728e+07,1.841973e+07,NaN,616.656260,8.219125e+06,8.137203e+06,117.908727,803.359886,...,51.798980,36.725728,10.142921,-15.446980,-0.964310,NaN,41.005764,37.121951,40.074347,38.962038
1956,75971.945672,3.042471,2.597305e+07,1.963772e+07,NaN,649.287314,8.549108e+06,8.443142e+06,118.483173,819.689680,...,50.644761,36.728189,10.189149,-14.954914,-0.796134,NaN,42.264125,38.052439,40.704608,39.878633
1957,75730.959076,3.079790,2.706907e+07,2.055731e+07,NaN,667.170975,8.890253e+06,8.782301e+06,119.062562,836.456039,...,50.462612,36.718872,10.082829,-15.352770,-0.721141,NaN,43.568850,38.677111,41.919898,40.904302
1958,75460.866054,3.086919,2.813011e+07,2.092901e+07,NaN,710.760380,9.161303e+06,9.062800e+06,119.644116,860.093655,...,49.246998,36.713626,9.570507,-13.085432,-0.764316,NaN,44.337550,39.390165,42.089756,40.821711


In [31]:
# whether the country is in the eurozone
eurozone_countries = ["Austria", "Belgium", "Cyprus", "Estonia", "Finland", "France", "Germany", "Greece", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Portugal", "Slovakia", "Slovenia", "Spain"]
eurozone_countries

['Austria',
 'Belgium',
 'Cyprus',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Portugal',
 'Slovakia',
 'Slovenia',
 'Spain']

In [45]:
df = df_final.reset_index()
df.head()

variable,country,Date,Average annual hours worked by persons engaged,Average depreciation rate of the capital stock,Capital stock at constant 2011 national prices (in mil. 2011US$),Capital stock at current PPPs (in mil. 2011US$),Correlation between expenditure shares of the country and the US (benchmark observations only),"Exchange rate, national currency/USD (market+estimated)",Expenditure-side real GDP at chained PPPs (in mil. 2011US$),Expenditure-side real GDP at current PPPs (in mil. 2011US$),...,Share of household consumption at current PPPs,Share of labour compensation in GDP at current national prices,Share of merchandise exports at current PPPs,Share of merchandise imports at current PPPs,Share of residual trade and GDP statistical discrepancy at current PPPs,"Statistical capacity indicator (source: World Bank, developing countries only)",TFP at constant national prices (2011=1),TFP level at current PPPs (USA=1),Welfare-relevant TFP at constant national prices (2011=1),Welfare-relevant TFP levels at current PPPs (USA=1)
0,Albania,1970,NaN,0.039577,9863.742188,8341.498047,NaN,5.0,7195.136230,6685.763672,...,0.725805,NaN,0.003096,-0.005470,-0.000216,NaN,NaN,NaN,NaN,NaN
1,Albania,1971,NaN,0.040581,14497.030273,9139.708008,NaN,5.0,7484.070801,6935.014160,...,0.723111,NaN,0.004856,-0.005447,-0.003275,NaN,NaN,NaN,NaN,NaN
2,Albania,1972,NaN,0.040550,15750.315430,9979.962891,NaN,5.0,7785.146973,7188.490234,...,0.718803,NaN,0.004268,-0.005906,-0.000823,NaN,NaN,NaN,NaN,NaN
3,Albania,1973,NaN,0.040338,17023.150391,10938.644531,NaN,5.0,8094.222656,7424.112305,...,0.715955,NaN,0.007400,-0.008299,-0.001680,NaN,NaN,NaN,NaN,NaN
4,Albania,1974,NaN,0.039625,18344.697266,12095.361328,NaN,5.0,8422.099609,7760.000977,...,0.710702,NaN,0.015429,-0.017236,0.001243,NaN,NaN,NaN,NaN,NaN


In [50]:
df['eurozone'] = [country in eurozone_countries for country in df['country']]

In [57]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print_full(df.groupby('country').sum().eurozone)

country
Albania                                       0.0
Algeria                                       0.0
Angola                                        0.0
Anguilla                                      0.0
Antigua and Barbuda                           0.0
Argentina                                     0.0
Armenia                                       0.0
Aruba                                         0.0
Australia                                     0.0
Austria                                      62.0
Azerbaijan                                    0.0
Bahamas                                       0.0
Bahrain                                       0.0
Bangladesh                                    0.0
Barbados                                      0.0
Belarus                                       0.0
Belgium                                      62.0
Belize                                        0.0
Benin                                         0.0
Bermuda                                   

In [63]:
df.to_csv("cleaned_dataset.csv", index = False)

In [4]:
df2 = pd.read_csv("cleaned_dataset.csv", low_memory=False)
df2

,country,Date,Average annual hours worked by persons engaged,Average depreciation rate of the capital stock,Capital stock at constant 2011 national prices (in mil. 2011US$),Capital stock at current PPPs (in mil. 2011US$),Correlation between expenditure shares of the country and the US (benchmark observations only),"Exchange rate, national currency/USD (market+estimated)",Expenditure-side real GDP at chained PPPs (in mil. 2011US$),Expenditure-side real GDP at current PPPs (in mil. 2011US$),...,Share of labour compensation in GDP at current national prices,Share of merchandise exports at current PPPs,Share of merchandise imports at current PPPs,Share of residual trade and GDP statistical discrepancy at current PPPs,"Statistical capacity indicator (source: World Bank, developing countries only)",TFP at constant national prices (2011=1),TFP level at current PPPs (USA=1),Welfare-relevant TFP at constant national prices (2011=1),Welfare-relevant TFP levels at current PPPs (USA=1),eurozone
0,Albania,1970,NaN,0.039577,9863.742188,8341.498047,NaN,5.000000,7195.136230,6685.763672,...,NaN,0.003096,-0.005470,-0.000216,NaN,NaN,NaN,NaN,NaN,False
1,Albania,1971,NaN,0.040581,14497.030273,9139.708008,NaN,5.000000,7484.070801,6935.014160,...,NaN,0.004856,-0.005447,-0.003275,NaN,NaN,NaN,NaN,NaN,False
2,Albania,1972,NaN,0.040550,15750.315430,9979.962891,NaN,5.000000,7785.146973,7188.490234,...,NaN,0.004268,-0.005906,-0.000823,NaN,NaN,NaN,NaN,NaN,False
3,Albania,1973,NaN,0.040338,17023.150391,10938.644531,NaN,5.000000,8094.222656,7424.112305,...,NaN,0.007400,-0.008299,-0.001680,NaN,NaN,NaN,NaN,NaN,False
4,Albania,1974,NaN,0.039625,18344.697266,12095.361328,NaN,5.000000,8422.099609,7760.000977,...,NaN,0.015429,-0.017236,0.001243,NaN,NaN,NaN,NaN,NaN,False
5,Albania,1975,NaN,0.039137,19702.917969,13168.867188,NaN,5.000000,8762.291992,8096.296387,...,NaN,0.018435,-0.017709,-0.002506,NaN,NaN,NaN,NaN,NaN,False
6,Albania,1976,NaN,0.038608,21122.320312,14067.861328,NaN,5.000000,9096.467773,8368.812500,...,NaN,0.013966,-0.011031,-0.009161,NaN,NaN,NaN,NaN,NaN,False
7,Albania,1977,NaN,0.038145,22577.404297,15281.502930,NaN,5.000000,9488.245117,8739.588867,...,NaN,0.018729,-0.014708,-0.007877,NaN,NaN,NaN,NaN,NaN,False
8,Albania,1978,NaN,0.037274,24115.947266,16651.291016,NaN,5.000000,9868.458984,9075.011719,...,NaN,0.014091,-0.016145,0.002439,NaN,NaN,NaN,NaN,NaN,False
9,Albania,1979,NaN,0.037119,25756.097656,18108.474609,NaN,6.167123,10175.361328,9360.236328,...,NaN,0.044798,-0.030448,-0.027609,NaN,NaN,NaN,NaN,NaN,False


In [6]:
# whether the country is in the eurozone
eurozone_countries = ["Austria", "Belgium", "Cyprus", "Estonia", "Finland", "France", "Germany", "Greece", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Portugal", "Slovakia", "Slovenia", "Spain"]
eurozone_countries

['Austria',
 'Belgium',
 'Cyprus',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Portugal',
 'Slovakia',
 'Slovenia',
 'Spain']

In [12]:
df = pd.read_excel('pwt90.xlsx', sheetname='Data')
df['eurozone'] = [country in eurozone_countries for country in df['country']]

In [11]:
tmp = df[df['eurozone']].country.unique()
tmp.sort()
print(tmp)

['Austria' 'Belgium' 'Cyprus' 'Estonia' 'Finland' 'France' 'Germany'
 'Greece' 'Ireland' 'Italy' 'Latvia' 'Lithuania' 'Luxembourg' 'Malta'
 'Netherlands' 'Portugal' 'Slovakia' 'Slovenia' 'Spain']


In [13]:
df.to_csv("cleaned_dataset.csv", index = False)